In [55]:
from requests import get
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep
from time import time
from random import randint
from IPython.core.display import clear_output
import math
import matplotlib.pyplot as plt

start_year =int(input("Enter starting year:")) 
end_year = int(input("Enter last year:"))+1

pages = [str(i) for i in range(1,5)]
years_url = [str(i) for i in range(start_year,end_year)]

names = []
years = []
imdb_ratings = []
metascores = []
votes = []
description = []

start_time = time()
requests = 0
max_requests = 4*(end_year-start_year)

for year in years_url:
    
    for page in pages:        
        response = get('http://www.imdb.com/search/title?release_date='+year+'&sort=num_votes,desc&page='+page)
        sleep(randint(8,15))
        
        requests+=1
        elapsed_time = time() - start_time
        print('Request:{}; Frequency:{} requests/s; Time:{} min {} s'.format(requests,requests/elapsed_time,int(elapsed_time/60),math.ceil(elapsed_time%60)))
        clear_output(wait = True)
        
        if response.status_code != 200:
            warn('Request{}; Status code:{}'.format(requests,response.status_code))
            
        if requests>max_requests:
            warn('Number of requests was greater than expected.')
            break
            
        page_html = BeautifulSoup(response.text,'html.parser')
        mv_containers = page_html.find_all('div',class_='lister-item mode-advanced')
        
        for container in mv_containers:
            
            if container.find('div',class_= 'ratings-metascore') is not None:
                
                name = container.h3.a.text
                names.append(name)
                
                year = container.h3.find('span', class_ = 'lister-item-year').text
                years.append(year)
            
                imdb = float(container.strong.text)
                imdb_ratings.append(imdb)
            
                m_score = container.find('span', class_ = 'metascore').text
                metascores.append(int(m_score))
                
                vote = container.find('span', attrs = {'name':'nv'})['data-value']
                votes.append(int(vote))
                
                desc = container.find_all('p',class_='text-muted')
                description.append(desc[1].text[5:])


movie_ratings = pd.DataFrame({'movie': names,
'year': years,
'imdb': imdb_ratings,
'metascore': metascores,
'votes': votes,
'description':description
})

print(movie_ratings.info())
movie_ratings.head(10)
movie_ratings.to_csv('movie_ratings.csv')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 142 entries, 0 to 141
Data columns (total 6 columns):
movie          142 non-null object
year           142 non-null object
imdb           142 non-null float64
metascore      142 non-null int64
votes          142 non-null int64
description    142 non-null object
dtypes: float64(1), int64(2), object(3)
memory usage: 6.8+ KB
None


,movie,year,imdb,metascore,votes,description
0,Gladiator,(2000),8.5,67,1300325,A former Roman General sets out to exact venge...
1,Memento,(2000),8.4,80,1092026,A man with short-term memory loss attempts to ...
2,Snatch,(2000),8.3,55,762775,"Unscrupulous boxing promoters, violent bookmak..."
3,Requiem for a Dream,(2000),8.3,68,744409,The drug-induced utopias of four Coney Island ...
4,X-Men,(2000),7.4,64,559894,In a world where mutants (evolved super-powere...
5,Cast Away,(2000),7.8,73,507478,A FedEx executive undergoes a physical and emo...
6,American Psycho,(2000),7.6,64,468980,A wealthy New York City investment banking exe...
7,Unbreakable,(2000),7.3,62,378505,A man learns something extraordinary about him...
8,Mission: Impossible II,(2000),6.1,59,306969,IMF agent Ethan Hunt is sent to Sydney to find...
9,Meet the Parents,(2000),7.0,73,305194,Male nurse Greg Focker meets his girlfriend's ...


In [53]:
movie_ratings.to_csv('Movie_ratings
                     .csv')

["Tyler Rake, a fearless black market mercenary, embarks on the most deadly extraction of his career when he's enlisted to rescue the kidnapped son of an imprisoned international crime lord.",
 'After splitting with the Joker, Harley Quinn joins superheroes Black Canary, Huntress and Renee Montoya to save a young girl from an evil crime lord.',
 "When Cecilia's abusive ex takes his own life and leaves her his fortune, she suspects his death was a hoax. As a series of coincidences turn lethal, Cecilia works to prove that she is being hunted by someone nobody can see.",
 'Miami detectives Mike Lowrey and Marcus Burnett must face off against a mother-and-son pair of drug lords who wreak vengeful havoc on their city.',
 'Two elven brothers embark on a quest to bring their father back for one day.',
 'After discovering a small, blue, fast hedgehog, a small-town police officer must help him defeat an evil genius who wants to do experiments on him.',
 'A covert team of immortal mercenaries ar